In [1]:
import os
import pyomo.environ as pyo
# Import the Dwave packages dimod and neal
import dimod
import neal
# Import Matplotlib to generate plots
import matplotlib.pyplot as plt
# Import numpy and scipy for certain numerical calculations below
import numpy as np
import math
from collections import Counter
import pandas as pd
from itertools import chain
import time
import networkx as nx
import pickle
from matplotlib import ticker

import sys

sys.path.append("../../src")

import stochastic_benchmark as SB
import bootstrap
import interpolate
import stats
from utils_ws import *

## Simulated Annealing (S.A.)

S.A. is an optimization algorithm that works by navigating the input space and evaluating the objective function (energy) similarly to hill climbing. The main difference is that S.A. implements a strategy to avoid getting stuck in local optima.

The general idea is to allow moves to a worse position, but in a structured way. When a candidate solution is selected, the probability of actually moving to the new position is determined by the change in the objective function and by a metric called **temperature**. When the temperature is high, the chance of performing a jump to a worse state increases, and the opposite happens when the temperature is low. By continuously decreasing the temperature during the execution of the algorithm, it can escape local optima during the first stages (high temperature), but it settles into a position towards the end of the execution (low temperature).

### Probability of Making a Jump

**Case 1:**
$$
\Delta E = E_{current} - E_{next} > 0
$$

It is always advantageous to move to a state with a better objective function. In this case, the probability is $P = 1$.

**Case 2:**
$$
\Delta E = E_{current} - E_{next} < 0
$$

In this case, the probability of taking the step is determined by:

$$
P = e^{-|\Delta E| / T}
$$

**Note:** This formulation applies to a minimization problem. To solve a maximization problem, the only change is that $\Delta E = E_{next} - E_{current}$.

### Temperature Decrease

The way the temperature decreases during the algorithm's execution varies between implementations. This is referred to as the temperature **schedule**, which is a function of time or the number of iterations.

**Note:** $\beta = 1 / T$ is also commonly used in the implementation of the algorithm.

Let's define a larger model, with 100 variables and random weights.

Assume that we are interested at the instance created with random weights $h_{i}, J_{i, j} \sim U[-1, +1]$.

### Problem statement

We pose the Ising problem as the following optimization problem:

$$
\min_{s \in \{ \pm 1 \}^n} H(s) = \min_{s \in \{ \pm 1 \}^n} \sum_{(i, j) \in E(G)} J_{i,j}s_is_j + \sum_{i \in V(G)} h_is_i + \beta
$$

where we optimize over spins $s \in \{ \pm 1 \}^n$, on a constrained graph $G(V,E)$, where the quadratic coefficients are $J_{i,j}$ and the linear coefficients are $h_i$.
We also include an arbitrary offset of the Ising model $\beta$.

### Loading The Data

The following cells load data from Simulated Annealing runs performed on a problem instance with variations in the number of reads parameter (1, 10, 1000).

These runs were originally executed using the [QuboNotebooks Repository](https://github.com/JuliaQUBO/QUBONotebooks/blob/main/notebooks_py/5-Benchmarking_python.ipynb).

In [2]:
current_path = os.getcwd()
pickle_path = os.path.join(current_path, 'results/')
if not(os.path.exists(pickle_path)):
    print('Results directory ' + pickle_path +
          ' does not exist. We will create it.')
    os.makedirs(pickle_path)
    !wget -O {pickle_path}results.zip -N -q "https://github.com/JuliaQUBO/QUBONotebooks/raw/main/notebooks_py/results.zip"

In [3]:
import zipfile
zip_name = os.path.join(pickle_path, 'results.zip')
overwrite_pickles = False
if os.path.exists(zip_name):
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(pickle_path)
    print('Results zip file has been extrated to ' + pickle_path)

Results zip file has been extrated to /home/jvpcms/purdue-internship/stochastic-benchmark/examples/Simulated_Annealing/results/


In [36]:
instance = 42    
results_name = "results_" + str(instance) + ".pkl"
results_name = os.path.join(pickle_path, results_name)

results = pickle.load(open(results_name, "rb"))

In [38]:
def print_dict_structure(d, indent=0):
    """
    Prints the keys of a nested dictionary in a file structure-like format.
    """
    for key, value in d.items():
        # Print the current key with indentation
        print(' ' * indent + str(key))
        
        # If the value is a dictionary, recurse into it
        if isinstance(value, dict):
            print_dict_structure(value, indent + 4)

# p
#     1
#         geometric
#     10
#         geometric
#     1000
#         geometric
# min_energy
#     geometric
# random_energy
#     geometric
# tts
#     1
#         geometric
#     10
#         geometric
#     1000
#         geometric
# ttsci
#     1
#         geometric
#     10
#         geometric
#     1000
#         geometric
# t
#     geometric
# best
#     1
#         geometric
#     10
#         geometric
#     1000
#         geometric
# bestci
#     1
#         geometric
#     10
#         geometric
#     1000
#         geometric

### Performance Metric

We are interested at the "aproximation ratio" metric. In this particular problem, it is calculated as follows:

- At first, we sample a subset from the input space and calculate the associated energies. Specifically 1000 random bitstrings, in this case.
- The best (minimum) energy found across the ramdom sample is denoted `minimum`, while the mean energy across the random sampling is denoted `random`.
- The aproximation ratio is calculated across an increasing number of sweeps of S.A. where `found` is the best energy found at the current moment in de algorithm's execution.

$$
approximation\space ratio = \frac{found - random}{minimum - random}
$$

Even though it is (very) unlikely that we found the best solution during the random sampling, this metric is useful because it gives us a general idea of how good our currently found solution is.


More details at [QuboNotebooks Repository](https://github.com/JuliaQUBO/QUBONotebooks/blob/main/notebooks_py/5-Benchmarking_python.ipynb).

In [19]:
here = os.getcwd()
parameter_names = [
    "sweeps",
    "reads",
]  # think about whether iterations should be a parameter or not.
instance_cols = [
    "instance"
]  # indicates how instances should be grouped, default is ['instance']

## Response information
response_key = "approx_ratio"  # Column with the response
response_dir = -1  # whether we want to maximize (1) or minimize (-1), default is 1

## Optimizations informations
recover = True  # Whether we want to read dataframes when available, default is True
reduce_mem = True  # Whether we want to segment bootstrapping and interpolation to reduce memory usage, default is True
smooth = True  # Whether virtual best should be monontonized, default is True

sb = SB.stochastic_benchmark(
    parameter_names=parameter_names,
    here=here,
    instance_cols=instance_cols,
    response_key=response_key,
    response_dir=response_dir,
    smooth=smooth,
)